In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
# nakon što stavimo datoteke na Git uvijek čitati u ovomo obliku! (štedi vrijeme)
import time
start = time.time()
df = pd.read_hdf('./data/raw/training_dataset.h5', 'df')
end = time.time()
print(end - start)

# nakon što napravimo promjene kako treba, exportat ću ih u novu datoteku pomoću naredbe
# df.to_hdf('training_dataset2.h5', key='df')

In [ ]:
# kasnije razmisliti o postavljanju na prosjek ovisno o vrsti klijenta? 
df['STAROST'].values[df['STAROST'].values >= 900] = -1
df['STAROST'].values[df['STAROST'].values < 0] = -1

In [ ]:
df['PRIJEVREMENI_RASKID'] = np.where(df['DATUM_ZATVARANJA']>=(df['PLANIRANI_DATUM_ZATVARANJA']- np.timedelta64(10, 'D')), 'N', df['PRIJEVREMENI_RASKID'])

In [ ]:
#brisanje outlier-a i ugovorenih iznosa = 0.0
df = df[(df['UGOVORENI_IZNOS'] < 2e5) & (df['UGOVORENI_IZNOS'] > 0.0)]

In [ ]:
### BRISANJE PARTIJA S VIŠE KLIJENATA
gb = df.groupby('OZNAKA_PARTIJE')['KLIJENT_ID'].nunique() # baci broj jedinstvenih klijenata za pojedinu partiju
# party_labels = df2['OZNAKA_PARTIJE'].unique().tolist() 
mask = gb[df['OZNAKA_PARTIJE']] == 1 # gb[df2['OZNAKA_PARTIJE']] vrati za svaku oznaku partije (svaki redak tablice) njenu vrijednost i usporedi je s 1
df = df[ mask.values ] # mask je neki objekt koji moram ovako koristiti, rezultat je skup gdje za svaku partiju ima jedan klijent

In [ ]:
df = df[df['PLANIRANI_DATUM_ZATVARANJA'] > df['DATUM_OTVARANJA']] 

In [ ]:
## POPUNJAVANJE MISSING VALUES UNAZAD (po grupama)

# Create a groupby object
grouped = df.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA'])

# Write a function that imputes median
def impute_backwards(series):
    return series.fillna(method = 'bfill')

# Impute age and assign to titanic['age']
df['NOVI_DATUM_ZATVARANJA', 'NOVA_VISINA_KAMATE'] = grouped['DATUM_ZATVARANJA', 'VISINA_KAMATE'].transform(impute_backwards)

In [ ]:
# neka tu stoji ako zatreba
# sortiranje, potrebno za filanje missing values
# df.sort_values(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA', 'DATUM_IZVJESTAVANJA', 'STANJE_NA_KRAJU_KVARTALA'], ascending=[True, True, True, True, False], inplace = True)

In [ ]:
# export datoteke
df.to_hdf('./data/raw/nakon_procesiranja.h5', key='df')